In [1]:
import os

In [2]:
os.getcwd()

'C:\\Users\\User\\Desktop\\demo'

In [3]:
# Specify the folder path containing the original images
folder_path = 'C:\\Users\\User\\Desktop\\demo\\Doubtful-1'

# Specify the folder path to save the updated images
save_folder_path = 'C:\\Users\\User\\Desktop\demo\\processed'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from skimage import io, img_as_float
from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage.util import random_noise

# Iterate over the files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith('.png') or filename.endswith('.jpg'):
        # Read the original image
        image_path = os.path.join(folder_path, filename)
        f = cv2.imread(image_path, 1)

        # Calculate and plot the histogram of the original image
        hist, bins = np.histogram(f.flatten(), bins=256, range=[0, 256])
        cdf = hist.cumsum()
        cdf_normalized = cdf / cdf.max()

        fig, ax = plt.subplots(figsize=(5, 5))
        ax.hist(f.flatten(), bins=256, range=[0, 256], color='r')
        ax.set_xlabel('pixel intensity')
        ax.set_ylabel('#pixels')
        ax.set_xlim(0, 255)

        ax2 = ax.twinx()
        ax2.plot(cdf_normalized, color='b')
        ax2.set_ylabel('cdf')
        ax2.set_ylim(0, 1)

        # Save the plot as an image in the specified directory
        #hist_image_path = os.path.join(save_folder_path, filename + '_hist.png')
        #plt.savefig(hist_image_path)
        plt.close()
        lab_img = cv2.cvtColor(f, cv2.COLOR_BGR2LAB)
        l, a, b = cv2.split(lab_img)

        # Perform histogram equalization (global)
        equ = cv2.equalizeHist(l)

        # Calculate and plot the histogram of the equalized image
        hist, bins = np.histogram(equ.flatten(), bins=256, range=[0, 256])
        cdf = hist.cumsum()
        cdf_normalized = cdf / cdf.max()

        fig, ax = plt.subplots(figsize=(5, 5))
        ax.hist(equ.flatten(), bins=256, range=[0, 256], color='r')
        ax.set_xlabel('pixel intensity')
        ax.set_ylabel('#pixels')
        ax.set_xlim(0, 255)

        ax2 = ax.twinx()
        ax2.plot(cdf_normalized, color='b')
        ax2.set_ylabel('cdf')
        ax2.set_ylim(0, 1)

        # Save the plot as an image in the specified directory
        #equ_image_path = os.path.join(save_folder_path, filename + '_equ.png')
        #plt.savefig(equ_image_path)
        #plt.close()

        # Perform fast non-local means denoising
        image = img_as_float(equ, force_copy=True)
        sigma = 0.03
        noisy_image = random_noise(image, var=sigma ** 2)
        sigma_est = np.mean(estimate_sigma(noisy_image))
        patch_size = 4
        patch_distance = 13
        denoised_image = denoise_nl_means(noisy_image, h=0.8 * sigma_est, fast_mode=True,
                                          patch_size=patch_size, patch_distance=patch_distance)

        # Save the denoised image in the specified directory
        denoised_image_path = os.path.join(save_folder_path, filename + '_denoised.png')
        denoised_image = (denoised_image * 255).astype(np.uint8)
        cv2.imwrite(denoised_image_path, denoised_image)


C:\Users\User\AppData\Local\Temp\ipykernel_11400\455786088.py:21: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(figsize=(5, 5))
C:\Users\User\AppData\Local\Temp\ipykernel_11400\455786088.py:47: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(figsize=(5, 5))
